# Portfolio Yml

In [1]:
import pandas as pd
from yaml import SafeDumper
import yaml
import intake

In [2]:
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = "{:,.2f}".format

In [3]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"
FILE = "fake_data.xlsx"

In [4]:
catalog = intake.open_catalog("./project_prioritization/*.yml")

In [5]:
# Read original df 
df_statewide = pd.read_excel(f"{GCS_FILE_PATH}{FILE}", sheet_name="fake")

In [6]:
# Fill in empty county names with various
df_statewide["full_county_name"] = df_statewide["full_county_name"].fillna("Various")

In [7]:
# Filter for unique full county names
toc = df_statewide[['full_county_name']].drop_duplicates().sort_values('full_county_name').reset_index(drop = True)

In [8]:
# Add a column for County
toc['County'] = ' County'

In [9]:
# Combine two columns into a new one for chapter title
toc['chapter_title'] = toc['full_county_name'] + toc['County']

In [10]:
# Change various
toc['chapter_title'] = toc['chapter_title'].replace({'Various County':'Various Counties'})

In [11]:
# Drop County col
toc = toc.drop(columns = ['County'])

In [12]:
# Create a sorted, unique list of all the counties in dataset
parameter_counties = df_statewide.full_county_name.sort_values().unique().tolist()

In [13]:
# Open my test yml as called "test" and load it as "analyses_data"
with open('../project_prioritization/test.yml') as test:
    analyses_data = yaml.load(test, yaml.Loader)

In [14]:
# Empty chapter list
chapters_list = []

In [15]:
# For every county in my list of counties, loop through and create
# chapters and parameters
for county in parameter_counties: 
    chapter_dict = {} 
    chapter_dict['chapters'] = None
    chapter_dict['params'] = {
                              'parameter_county': f''{county}'',
                               'county_title': f''{county} County'',}
    chapters_list += [chapter_dict]

In [16]:
parts_list = [{'parts': chapters_list}]

In [17]:
output = yaml.dump(parts_list)

In [18]:
with open('../project_prioritization/test.yml', 'w') as test:
       test.write(output)